# Pyrolysis Data Extraction & Integration Template
This notebook helps you extract, structure, and load plastic pyrolysis data from existing literature into a PostgreSQL database.

Follow these steps for each new research paper or dataset you extract.

In [ ]:
# STEP 1: Setup database connection
from sqlalchemy import create_engine
import pandas as pd

# Replace with your actual credentials
USER = 'your_username'
PASSWORD = 'your_password'
HOST = 'localhost'
PORT = '5432'
DB = 'pyrolysis_db'

engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}')

## STEP 2: Load and prepare your extracted data
Replace the paths with the correct CSV or Excel files from your extraction.
Ensure they are properly cleaned, and the units match the schema.

In [ ]:
# Example of loading extracted tables
feedstock = pd.read_csv('plastic_feedstock_extracted.csv')
blends = pd.read_csv('blend_compositions_extracted.csv')
conditions = pd.read_csv('pyrolysis_conditions_extracted.csv')
outputs = pd.read_csv('pyrolysis_outputs_extracted.csv')

## STEP 3: Validate and clean your data
Examples:
- Ensure blend percentages sum to 100
- Normalize plastic IDs
- Check for nulls in key columns

In [ ]:
# Sample validation
assert all(abs(blends[['HDPE', 'LDPE', 'PP', 'PS']].sum(axis=1) - 100) < 0.5), "Blend percentages must sum to 100%"
feedstock.drop_duplicates(subset='plastic_id', inplace=True)

## STEP 4: Upload to PostgreSQL
Ensure your schema is already created in the database.
You can use `if_exists='append'` to load new data without deleting old entries.

In [ ]:
# Load into the database
feedstock.to_sql('plastic_feedstock', con=engine, if_exists='append', index=False)
blends.to_sql('blend_compositions', con=engine, if_exists='append', index=False)
conditions.to_sql('pyrolysis_conditions', con=engine, if_exists='append', index=False)
outputs.to_sql('pyrolysis_outputs', con=engine, if_exists='append', index=False)

## STEP 5: Query and confirm the upload
Check that the data was uploaded correctly.

In [ ]:
# Sample query
df_check = pd.read_sql_query("""
SELECT o.*, c.temperature, c.catalyst, b.HDPE, b.LDPE, b.PP, b.PS
FROM pyrolysis_outputs o
JOIN pyrolysis_conditions c ON o.condition_id = c.condition_id
JOIN blend_compositions b ON c.blend_id = b.blend_id
""", engine)

df_check.head()